In [2]:
!pip install gym

In [3]:
import torch
from torch import nn
from torch import optim
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from gym.spaces import Discrete, Box

In [4]:
print(f"torch version:\t{torch.__version__}")
print(f"gym version:\t{gym.__version__}")

torch version:	1.7.0
gym version:	0.18.0


`Categorical`'s `probs` argument takes in a tensor of 'probabilities' in range `[0, inf)` ie. non-negative but does not need to sum to 1, as the class will automatically normalize the values to make the distribution. Make sure to sigmoid or softmax activations before passing this argument.

`Categorical`'s `logits` argument takes a tensor of values in range `(-inf, inf)` and will turn it into a probability distribution that sums to 1, probably with softmax but idk.

In [21]:
probs_list = [0.25, 0.25, 0.21, 0.80]
dist = torch.distributions.categorical.Categorical(probs=torch.tensor(probs_list))
classes = np.zeros(len(probs))
iterations = 10000
for i in range(iterations):
    class_idx = dist.sample().item()
    classes[class_idx] += 1
print(f"{classes/iterations}")

[0.1628 0.1733 0.1379 0.526 ]


In [22]:
logits_list = [-1.05, -0.15, 0.41, 1.20]
dist = torch.distributions.categorical.Categorical(logits=torch.tensor(logits_list))
classes = np.zeros(len(probs))
iterations = 10000
for i in range(iterations):
    class_idx = dist.sample().item()
    classes[class_idx] += 1
print(f"{classes/iterations}")

[0.0597 0.1389 0.2548 0.5466]


In [26]:
dist.probs, dist.logits

(tensor([0.0580, 0.1426, 0.2496, 0.5499]),
 tensor([-2.8480, -1.9480, -1.3880, -0.5980]))

## Define Model

In [ ]:
class Model:
    def __init_(self, env):
        self.n_inputs = env.observation_space.shape[0]
        self.n_outputs = env.action_space.n
        
        # Define neural network
        self.network = nn.Sequential(
            nn.Linear(self.n_inputs, 32),
            nn.ReLU(),
            nn.Linear(32, self.n_outputs),
            nn.Softmax(dim=-1)
        )
    
    def predict(self, inputs):
        action_probs = self.network(torch.FloatTensor(state))
        return action_probs

In [ ]:
def CreateMLP(sizes, hidden_act_fn=nn.ReLU, output_act_fn=nn.Softmax):
    # Sizes consists of the sizes of all layers
    layers = []
    for i in range(len(sizes)-1):
        

## Finite-Horizon Discounted Rewards helper function

In [95]:
def discounted_rewards(rewards, gamma=0.99):
    # rewards must be a rank 1 array of scalar reward values
    assert len(rewards.shape) == 1
    trajec_len = rewards.size # Number of reward values
    discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
    for i in reversed(range(trajec_len)):
        discounted_rewards[i] = rewards[i] + gamma * (discounted_rewards[i+1] if i+1 < trajec_len else 0)
    return discounted_rewards

In [96]:
rewards = np.random.randint(6, size=(12,))
rewards

array([2, 0, 4, 2, 1, 3, 2, 4, 4, 4, 1, 4])

In [97]:
discounted_rewards(rewards, gamma=1)

array([31., 29., 29., 25., 23., 22., 19., 17., 13.,  9.,  5.,  4.],
      dtype=float32)

## Training Loop

In [ ]:
def train(env, model, num_episodes=150, ep_batch_size, gamma=0.99):
    